### Genome-Scale Metabolic Modeling
* Metabolic fluxes were calculated using the COBRApy software and Gurobi Optimizer.
* iML1515 E. coli K-12 MG1655 metabolic model was utilized to simulate individual gene deletion under various carbon conditions.

We begin with importing the necessary files & packages

* python $\;\;\;\;$ :  v. 3.7
* gurobipy $\;$ :  v. 9.1.2
* cobra    $\;\;\;\;\;\;$    :  v. 0.22.1
* numpy  $\;\;\;\;$     : v. 1.21.6
* optlang $\;\;\;$ : v.  1.5.0

In [ ]:
#Python dependencies

import pandas as pd # v. 1.1.5
import cobra # v. 0.22.1
import numpy as np # v. 1.21.6
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

**Model modification**
 * Metabolic reactions associated with oxalacetate uptake were previously not included in the model.
  - iML1515 was updated based on data from EcoCyc (https://ecocyc.org/) and BiGG (http://bigg.ucsd.edu/).
 
 
 * We added 3 new reactions(EX_oaa_e, OAAtex, OAAt2_2pp) to the iML1515 model.
 
 1) EX_oaa_e
  - Reaction: oaa_e -->
  - Name: Oxaloacetate exchange
  - Subsystem: Extracellular exchange
  - Gene reaction rule: None
 
 2) OAAtex
  - Reaction: oaa_e <=> oaa_p
  - Name: Oxaloacetate transport via diffusion extracellular to periplasm
  - Subsystem: Transport Outer Membrane Porin
  - Gene reaction rule: b0241 or b0929 or b2215
 
 3) OAAt2_2pp
  - Reaction: 2 h_p + oaa_p <=> 2 h_c + oaa_c
  - Name: Oxaloacetate transport via proton symport 2 H periplasm
  - Subsystem: Transport Inner Membrane
  - Gene reaction rule: b3528
 
 
 * We added 2 new metabolites (oaa_e, oaa_p) to the iML1515 model.
 
 1) oaa_e
  - Formula: C4H2O5
  - Name: Oxaloacetate
  - Charge = -2
  - Compartment: e
 
 2) oaa_p
  - Formula: C4H2O5
  - Name: Oxaloacetate
  - Charge = -2
  - Compartment: p


 * Updated iML1515 was named by iML1515_oaa

In [ ]:
"""
Function
"""

# check component of model
# model: Variable of target model (e.g,. iML1515)
# summary (opt.): if True, run model.summary() (default: False).
def check_model (model, summary = False):
    print ('model name: {0}'.format(model.id))
    print ('{0} Reactions: {1}'.format(model.id, len(model.reactions)))
    print ('{0} Genes: {1}'.format(model.id, len(model.genes)))
    print ('{0} Metabolites: {1}'.format(model.id, len(model.metabolites)))

    if summary:
        print (model.summary())


"""
Load iML1515
"""
iML1515 = cobra.io.read_sbml_model('iML1515.xml')

# Check component of iML1515
check_model (iML1515)


"""
Make a new metabolites
"""
# make oxaloacetate in extracellular
oaa_e = cobra.Metabolite('oaa_e',
                         formula = 'C4H2O5',
                         name = 'Oxaloacetate',
                         charge = -2,
                         compartment = 'e')

# make oxaloacetate in periplasm
oaa_p = cobra.Metabolite('oaa_p',
                         formula = 'C4H2O5',
                         name = 'Oxaloacetate',
                         charge = -2,
                         compartment = 'p')


"""
Load metabolites
"""

# load H+ in periplasm
h_p = iML1515.metabolites.get_by_id('h_p')

# load H+ in cytosol
h_c = iML1515.metabolites.get_by_id('h_c')

# load oxaloacetate in cytosol
oaa_c = iML1515.metabolites.get_by_id('oaa_c')


"""
Make new reactions
"""

# make oxaloacetate exchange reaction
EX_oaa_e = cobra.Reaction('EX_oaa_e')
EX_oaa_e.name = 'Oxaloacetate exchange'
EX_oaa_e.subsystem = 'Extracellular exchange'
EX_oaa_e.bounds = (0.0, 1000.0)
EX_oaa_e.add_metabolites({oaa_e: -1.0})

# make oxaloacetate transport via diffusion extracellular to periplasm reaction
OAAtex = cobra.Reaction('OAAtex')
OAAtex.name = 'Oxaloacetate transport via diffusion extracellular to periplasm'
OAAtex.subsystem = 'Transport Outer Membrane Porin'
OAAtex.bounds = (-1000.0, 1000.0)
OAAtex.add_metabolites({oaa_e: -1.0,
                        oaa_p: 1.0})
OAAtex.gene_reaction_rule = 'b0241 or b0929 or b2215'

# make oxaloacetate transport via proton symport 2 H periplasm
OAAt2_2pp = cobra.Reaction('OAAt2_2pp')
OAAt2_2pp.name = 'Oxaloacetate transport via proton symport 2 H periplasm'
OAAt2_2pp.subsystem = 'Transport Inner Membrane'
OAAt2_2pp.bounds = (-1000.0, 1000.0)
OAAt2_2pp.add_metabolites({h_p: -2.0,
                        oaa_p: -1.0,
                        h_c: 2.0,
                        oaa_c: 1.0})
OAAt2_2pp.gene_reaction_rule = 'b3528'


"""
Add new reactions (EX_oaa_e, OAAtex, and OAAt2_2pp) to iML1515
"""
iML1515.add_reactions([EX_oaa_e, OAAtex, OAAt2_2pp])

# check component of updated iML1515
check_model(iML1515)


"""
Output updated iML1515 name by iML1515_oaa
"""
iML1515.id = 'iML1515_oaa'
cobra.io.write_sbml_model(iML1515, 'iML1515_oaa.xml')

**Model parameter setting**

In [ ]:
#Upload model
model =cobra.io.read_sbml_model("iML1515_oaa.xml") 

# Assign solver
model.solver = "gurobi" 

# Objective function
model.objective = 'BIOMASS_Ec_iML1515_core_75p37M' 

# Simulation type, 1: FBA,  2: pFBA, 3: MOMA
fba_model_sim_type = 3 

#Flux repeat
repetition = 3 # The average flux across repeated samples will be used to represent each reaction's representative flux 

**carbon condition setting**

| **Carbon** |    **acetate**   |  **adenosine**  | **D-alanine** |  **fructose**  |   **fucose**  |  **fumarate** | **galactose** | **galacturonate** | **gluconate** |      **glucosamine**     |
|:----------:|:----------------:|:---------------:|:-------------:|:--------------:|:-------------:|:-------------:|:-------------:|:-----------------:|:-------------:|:------------------------:|
|  file name |        ac        |       adn       |      alaD     |       fru      |      fuc      |      fum      |      gal      |       galur       |      glcn     |            gam           |
| **Carbon** |    **glucose**   | **glucuronate** |  **glycerol** |   **lactate**  | **L-alanine** |   **malate**  |  **maltose**  |    **mannitol**   |  **mannose**  | **N-acetyl glucosamine** |
|  file name |        glc       |      glcur      |      glyc     |       lac      |      alaL     |      mal      |      malt     |        mnl        |      man      |           acgam          |
| **Carbon** | **oxaloacetate** |   **pyruvate**  |  **ribose**  | **saccharate** |  **sorbitol** | **succinate** | **thymidine** |   **trehalose**   |   **xylose**  |    **a-ketoglutarate**   |
|  file name |        oaa       |       pyr       |      rib      |      sacc      |      sbt      |      succ     |     thymd     |        tre        |      xyl      |            akg           |


We first choose the carbon condition in which the simulation would be conducted. 

In [ ]:
# The carbon condition input must be equal to the designated name format.
# ex) glc (o), glucose(x)
carbon_condition = "glc"


**Model simulation**

* Run the code below for simulating various gene deletions.
* Designate genes subjected for deletion through "util/gene_deletion.csv".
* Change the initial reactions bound through "util/default_setting.csv". 


In [ ]:
#Designate simulation method
def fba_model_simulation(model):
    if fba_model_sim_type ==1:
        return  model.optimize()
    elif fba_model_sim_type ==2:
        return  cobra.flux_analysis.parsimonious.pfba(model)
    elif fba_model_sim_type ==3:
        return cobra.flux_analysis.moma(model)

ex_rxn_df = pd.read_csv("util/carbon_rxn.txt", sep="\t")
rxn_dict = {ex_rxn_df.iloc[i,0]: ex_rxn_df.iloc[i,1] for i in ex_rxn_df.index}

#Initial model setting
default_para = pd.read_csv("util/default_setting.csv")
gene_ko_para = pd.read_csv("util/gene_deletion.csv")
cobra_config = cobra.Configuration().processes
dataframe_fluxes  = pd.DataFrame()

#For default setting in the model
for index in default_para.index:
    default_para_id = default_para.iat[index,0]
    default_para_urxn = default_para.iat[index,2]
    default_para_lrxn = default_para.iat[index,1]
    model.reactions.get_by_id(default_para_id).upper_bound = default_para_urxn
    model.reactions.get_by_id(default_para_id).lower_bound = default_para_lrxn

model.reactions.get_by_id(rxn_dict[carbon_condition]).upper_bound = 1000
model.reactions.get_by_id(rxn_dict[carbon_condition]).lower_bound = -10

pfba_solution = fba_model_simulation(model)
flux_simul = pfba_solution.fluxes
flux_simul.name = "negative control"
#dataframe_fluxes = dataframe_fluxes.append(flux_simul)
column_index = flux_simul.shape[0]
gene_error_list  = []
error_flux =  pd.Series([""])
error_flux = error_flux.repeat(column_index)
error_flux.name = "default"
error_flux.index = flux_simul.index


#gene deletion simulation
for index in gene_ko_para.index:
    model_indiv = model.copy()
    gene_ko_para_id = gene_ko_para.iat[index, 0]
    model_indiv.genes.get_by_id(gene_ko_para_id).knock_out() 
    temp_flux =pd.Series([0])
    temp_flux = temp_flux.repeat(column_index)
    temp_flux.index = flux_simul.index
    error_foot  =0

    for rep in range(repetition):

        try:
            pfba_solution = fba_model_simulation(model_indiv)

        except cobra.exceptions.Infeasible: #some gene deletions lead to infeasible simulations
            gene_error_list.append(gene_ko_para_id) # Therefore these data are discarded
            error_foot=1
            #error_flux2 = error_flux.copy()
            #error_flux2.name = gene_ko_para_id
            #dataframe_fluxes = dataframe_fluxes.append(error_flux2)


        else:
            # pfba_solution = model_indiv.optimize()
            # print(pfba_solution.fluxes["BIOMASS_Ec_iML1515_core_75p37M"])
            # print(model_indiv.summary())
            temp_flux = temp_flux.add(pfba_solution.fluxes)

    flux_simul = temp_flux/repetition
    flux_simul.name = gene_ko_para_id

    if error_foot ==0:
        dataframe_fluxes = dataframe_fluxes.append(flux_simul)

gene_error_list = list(set(gene_error_list))
print(f"The infeasible genes are:{gene_error_list}")

#Export to feather file
dataframe_fluxes= dataframe_fluxes.reindex(temp_flux.index, axis=1)
dataframe_fluxes.to_feather("output/simulated_fluxes("+carbon_condition+").feather")